# MiniGPT: Implementação de um modelo Transformer simplificado para geração de texto
- Versão 2.0 - Corpus Melhorado e Prompts Alinhados
---
- Autor: MR Autoral
- Data: 2025
- Versão: 2.0.0 - Corpus expandido e prompts contextualizados
---

## SETUP INICIAL

In [1]:
import os
import logging
from typing import Optional, Tuple, Dict, Any
from dataclasses import dataclass, field
from pathlib import Path

import torch
import torch.nn as nn
from torch.nn import functional as F

import torch
import requests


### CONFIGURAÇÕES E HIPERPARÂMETROS

In [2]:
@dataclass
class ModelConfig:
    """Configurações do modelo MiniGPT"""
    # Hiperparâmetros de treinamento
    batch_size: int = 32
    block_size: int = 64
    max_iters: int = 3000
    eval_interval: int = 300
    learning_rate: float = 3e-4
    eval_iters: int = 200
    
    # Arquitetura do modelo
    n_embd: int = 128
    n_head: int = 8
    n_layer: int = 6
    dropout: float = 0.1
    
    # Configurações do sistema
    device: str = 'cuda'
    seed: int = 1337
    
    # Configurações de dados
    train_split: float = 0.9
    input_files: list[str] = field(default_factory=lambda: [
        'dados-doencas-textos-curtos.txt',
        'dados-doencas-textos-longos.txt',
        'dados-perguntas-respostas.txt'
    ])

In [3]:
class Logger:
    """Classe para logging estruturado"""
    
    def __init__(self, name: str = "MiniGPT"):
        self.logger = logging.getLogger(name)
        self.logger.setLevel(logging.INFO)
        
        if not self.logger.handlers:
            handler = logging.StreamHandler()
            formatter = logging.Formatter(
                '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
            )
            handler.setFormatter(formatter)
            self.logger.addHandler(handler)
    
    def info(self, message: str) -> None:
        self.logger.info(message)
    
    def warning(self, message: str) -> None:
        self.logger.warning(message)
    
    def error(self, message: str) -> None:
        self.logger.error(message)

## PREPARAÇÃO E PROCESSAMENTO DE DADOS

In [4]:
class TextDataProcessor:
    """Classe responsável pelo processamento de dados de texto"""
    
    def __init__(self, config: ModelConfig, logger: Logger):
        self.config = config
        self.logger = logger
        self.chars = []
        self.vocab_size = 0
        self.stoi = {}
        self.itos = {}
        self._is_initialized = False
        
    def create_rich_corpus(self, file) -> None:
        """Cria um corpus rico e extenso sobre tecnologia"""
        owner = "grupo6-tcc"
        repo = "mini-gpt"
        branch = "main"
        
        url = f"https://raw.githubusercontent.com/{owner}/{repo}/{branch}/{file}"
        response = requests.get(url)

        if response.status_code == 200:
            with open(file, 'w', encoding='utf-8') as f:
                f.write(response.text)
            self.logger.info(f"✓ Corpus rico criado: '{file}' ({len(response.text)} caracteres)")
        else:
            self.logger.info(f"✗ {file} não encontrado")
    
    def _build_vocabulary(self, text: str) -> None:
        """Constrói o vocabulário a partir do texto"""
        # Cria vocabulário
        self.chars = sorted(list(set(text)))
        self.vocab_size = len(self.chars)
        
        # Cria mapeamentos
        self.stoi = {ch: i for i, ch in enumerate(self.chars)}
        self.itos = {i: ch for i, ch in enumerate(self.chars)}
        
        self._is_initialized = True
        self.logger.info(f"Vocabulário criado com {self.vocab_size} caracteres únicos")
        self.logger.info(f"Amostra do vocabulário: {''.join(self.chars[:30])}{'...' if len(self.chars) > 30 else ''}")
    
    def load_and_process_data(self, input_file_index=0) -> Tuple[torch.Tensor, torch.Tensor]:
        """Carrega e processa os dados de texto"""
        # Verifica se o arquivo existe, senão cria um corpus rico
        for file in self.config.input_files:
            if not Path(file).exists():
                self.logger.warning(f"Arquivo '{file}' não encontrado")
                self.create_rich_corpus(file)
        
        # Carrega o texto
        try:
            with open(self.config.input_files[input_file_index], 'r', encoding='utf-8') as f:
                text = f.read()
        except Exception as e:
            self.logger.error(f"Erro ao carregar arquivo: {e}")
            raise
        
        if not text.strip():
            raise ValueError("Arquivo de texto está vazio")
        
        self.logger.info(f"Texto carregado: {len(text)} caracteres")
        
        # Constrói vocabulário
        self._build_vocabulary(text)
        
        # Tokeniza o texto
        try:
            data = torch.tensor(self.encode(text), dtype=torch.long)
        except Exception as e:
            self.logger.error(f"Erro na tokenização: {e}")
            raise
        
        # Divide em treino e validação
        n = int(self.config.train_split * len(data))
        train_data = data[:n]
        val_data = data[n:]
        
        self.logger.info(f"Dados divididos: {len(train_data)} treino, {len(val_data)} validação")
        
        return train_data, val_data
    
    def encode(self, text: str) -> list:
        """Codifica texto em lista de inteiros"""
        if not self._is_initialized:
            raise RuntimeError("Vocabulário não foi inicializado. Chame load_and_process_data() primeiro.")
        
        try:
            return [self.stoi[c] for c in text]
        except KeyError as e:
            missing_char = str(e).strip("'")
            self.logger.error(f"Caractere '{missing_char}' não encontrado no vocabulário")
            # Filtra caracteres desconhecidos
            return [self.stoi[c] for c in text if c in self.stoi]
    
    def decode(self, tokens: list) -> str:
        """Decodifica lista de inteiros em texto"""
        if not self._is_initialized:
            raise RuntimeError("Vocabulário não foi inicializado. Chame load_and_process_data() primeiro.")
        
        try:
            return ''.join([self.itos[i] for i in tokens])
        except KeyError as e:
            self.logger.error(f"Token {e} não encontrado no vocabulário")
            # Filtra tokens desconhecidos
            return ''.join([self.itos[i] for i in tokens if i in self.itos])

In [5]:
class DataLoader:
    """Classe para carregamento de lotes de dados"""
    
    def __init__(self, train_data: torch.Tensor, val_data: torch.Tensor, 
                 config: ModelConfig):
        self.train_data = train_data
        self.val_data = val_data
        self.config = config
        
        # Validações
        if len(train_data) < config.block_size:
            raise ValueError(f"Dados de treino muito pequenos: {len(train_data)} < {config.block_size}")
        if len(val_data) < config.block_size:
            raise ValueError(f"Dados de validação muito pequenos: {len(val_data)} < {config.block_size}")
    
    def get_batch(self, split: str) -> Tuple[torch.Tensor, torch.Tensor]:
        """Carrega um lote de dados"""
        data = self.train_data if split == 'train' else self.val_data
        
        # Garante que temos dados suficientes
        max_start_idx = len(data) - self.config.block_size
        if max_start_idx <= 0:
            raise ValueError(f"Dados insuficientes para block_size {self.config.block_size}")
        
        ix = torch.randint(max_start_idx, (self.config.batch_size,))
        x = torch.stack([data[i:i+self.config.block_size] for i in ix])
        y = torch.stack([data[i+1:i+self.config.block_size+1] for i in ix])
        
        x, y = x.to(self.config.device), y.to(self.config.device)
        return x, y

## COMPONENTES DO MODELO TRANSFORMER

In [6]:
class AttentionHead(nn.Module):
    """Uma única cabeça do mecanismo de autoatenção"""
    
    def __init__(self, config: ModelConfig, head_size: int):
        super().__init__()
        self.head_size = head_size
        self.config = config
        
        self.key = nn.Linear(config.n_embd, head_size, bias=False)
        self.query = nn.Linear(config.n_embd, head_size, bias=False)
        self.value = nn.Linear(config.n_embd, head_size, bias=False)
        
        self.register_buffer(
            'tril', 
            torch.tril(torch.ones(config.block_size, config.block_size))
        )
        self.dropout = nn.Dropout(config.dropout)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        B, T, C = x.shape
        
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)
        
        # Calcula scores de atenção
        wei = q @ k.transpose(-2, -1) * (self.head_size ** -0.5)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        
        # Agregação ponderada
        out = wei @ v
        return out

In [7]:
class MultiHeadAttention(nn.Module):
    """Múltiplas cabeças de autoatenção em paralelo"""
    
    def __init__(self, config: ModelConfig):
        super().__init__()
        head_size = config.n_embd // config.n_head
        
        self.heads = nn.ModuleList([
            AttentionHead(config, head_size) for _ in range(config.n_head)
        ])
        self.proj = nn.Linear(config.n_embd, config.n_embd)
        self.dropout = nn.Dropout(config.dropout)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [8]:
class FeedForward(nn.Module):
    """Rede feed-forward"""
    
    def __init__(self, config: ModelConfig):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(config.n_embd, 4 * config.n_embd),
            nn.GELU(),
            nn.Linear(4 * config.n_embd, config.n_embd),
            nn.Dropout(config.dropout),
        )
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.net(x)

In [9]:
class TransformerBlock(nn.Module):
    """Bloco Transformer completo"""
    
    def __init__(self, config: ModelConfig):
        super().__init__()
        self.sa = MultiHeadAttention(config)
        self.ffwd = FeedForward(config)
        self.ln1 = nn.LayerNorm(config.n_embd)
        self.ln2 = nn.LayerNorm(config.n_embd)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

## MODELO PRINCIPAL

In [10]:
class MiniGPT(nn.Module):
    """Modelo MiniGPT completo"""
    
    def __init__(self, config: ModelConfig, vocab_size: int):
        super().__init__()
        self.config = config
        self.vocab_size = vocab_size
        
        self.token_embedding_table = nn.Embedding(vocab_size, config.n_embd)
        self.position_embedding_table = nn.Embedding(config.block_size, config.n_embd)
        
        self.blocks = nn.Sequential(*[
            TransformerBlock(config) for _ in range(config.n_layer)
        ])
        
        self.ln_f = nn.LayerNorm(config.n_embd)
        self.lm_head = nn.Linear(config.n_embd, vocab_size)
        
        # Inicialização de pesos
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        """Inicialização de pesos do modelo"""
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    
    def forward(self, idx: torch.Tensor, targets: Optional[torch.Tensor] = None) -> Tuple[torch.Tensor, Optional[torch.Tensor]]:
        B, T = idx.shape
        
        # Validação de entrada
        if T > self.config.block_size:
            raise ValueError(f"Sequência muito longa: {T} > {self.config.block_size}")
        
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=self.config.device))
        
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        
        loss = None
        if targets is not None:
            B, T, C = logits.shape
            logits_flat = logits.view(B*T, C)
            targets_flat = targets.view(B*T)
            loss = F.cross_entropy(logits_flat, targets_flat)
        
        return logits, loss
    
    def generate(self, idx: torch.Tensor, max_new_tokens: int, temperature: float = 1.0) -> torch.Tensor:
        """Gera novos tokens de forma autoregressiva com controle de temperatura"""
        self.eval()
        
        with torch.no_grad():
            for _ in range(max_new_tokens):
                # Limita o contexto ao tamanho máximo
                idx_cond = idx[:, -self.config.block_size:]
                logits, _ = self(idx_cond)
                logits = logits[:, -1, :] / temperature
                probs = F.softmax(logits, dim=-1)
                idx_next = torch.multinomial(probs, num_samples=1)
                idx = torch.cat((idx, idx_next), dim=1)
        
        self.train()
        return idx
    
    def count_parameters(self) -> int:
        """Conta o número total de parâmetros"""
        return sum(p.numel() for p in self.parameters())

## TREINADOR DO MODELO

In [11]:
class ModelTrainer:
    """Classe responsável pelo treinamento do modelo"""
    
    def __init__(self, model: MiniGPT, data_loader: DataLoader, 
                 config: ModelConfig, logger: Logger):
        self.model = model
        self.data_loader = data_loader
        self.config = config
        self.logger = logger
        
        self.optimizer = torch.optim.AdamW(
            model.parameters(), 
            lr=config.learning_rate
        )
        
        self.training_history = {
            'train_loss': [],
            'val_loss': [],
            'iterations': []
        }
    
    @torch.no_grad()
    def estimate_loss(self) -> Dict[str, float]:
        """Estima a perda nos conjuntos de treino e validação"""
        out = {}
        self.model.eval()
        
        for split in ['train', 'val']:
            losses = torch.zeros(self.config.eval_iters)
            for k in range(self.config.eval_iters):
                try:
                    X, Y = self.data_loader.get_batch(split)
                    logits, loss = self.model(X, Y)
                    losses[k] = loss.item()
                except Exception as e:
                    self.logger.error(f"Erro na avaliação: {e}")
                    losses[k] = float('inf')
            out[split] = losses.mean().item()
        
        self.model.train()
        return out
    
    def train(self) -> Dict[str, Any]:
        """Executa o treinamento do modelo"""
        self.logger.info("Iniciando treinamento...")
        self.logger.info(f"Parâmetros do modelo: {self.model.count_parameters()/1e6:.2f}M")
        self.logger.info(f"Dispositivo: {self.config.device}")
        
        try:
            for iter_num in range(self.config.max_iters):
                # Avaliação periódica
                if iter_num % self.config.eval_interval == 0 or iter_num == self.config.max_iters - 1:
                    losses = self.estimate_loss()
                    self.logger.info(
                        f"Iteração {iter_num}: "
                        f"perda treino {losses['train']:.4f}, "
                        f"perda validação {losses['val']:.4f}"
                    )
                    
                    # Salva histórico
                    self.training_history['train_loss'].append(losses['train'])
                    self.training_history['val_loss'].append(losses['val'])
                    self.training_history['iterations'].append(iter_num)
                
                # Passo de treinamento
                try:
                    xb, yb = self.data_loader.get_batch('train')
                    logits, loss = self.model(xb, yb)
                    
                    self.optimizer.zero_grad(set_to_none=True)
                    loss.backward()
                    self.optimizer.step()
                    
                except Exception as e:
                    self.logger.error(f"Erro no passo de treinamento {iter_num}: {e}")
                    continue
            
            self.logger.info("Treinamento concluído!")
            
        except KeyboardInterrupt:
            self.logger.info("Treinamento interrompido pelo usuário")
        except Exception as e:
            self.logger.error(f"Erro durante o treinamento: {e}")
            raise
        
        return self.training_history

## GERADOR DE TEXTO

In [12]:
class TextGenerator:
    """Classe para geração de texto"""
    
    def __init__(self, model: MiniGPT, processor: TextDataProcessor, 
                 config: ModelConfig, logger: Logger):
        self.model = model
        self.processor = processor
        self.config = config
        self.logger = logger
    
    def generate_text(self, prompt: str, max_new_tokens: int = 300, temperature: float = 0.8) -> str:
        """Gera texto a partir de um prompt"""
        if not prompt:
            prompt = "A"  # Prompt mínimo
        
        self.logger.info(f"Gerando texto com prompt: '{prompt[:50]}{'...' if len(prompt) > 50 else ''}'")
        
        try:
            # Codifica o prompt
            encoded_prompt = self.processor.encode(prompt)
            if not encoded_prompt:
                self.logger.warning("Prompt vazio após codificação, usando 'A'")
                encoded_prompt = self.processor.encode("A")
            
            context = torch.tensor(
                [encoded_prompt], 
                dtype=torch.long, 
                device=self.config.device
            )
            
            # Gera tokens
            generated_tokens = self.model.generate(
                context, 
                max_new_tokens, 
                temperature=temperature
            )[0].tolist()
            
            # Decodifica para texto
            generated_text = self.processor.decode(generated_tokens)
            
            return generated_text
            
        except Exception as e:
            self.logger.error(f"Erro na geração de texto: {e}")
            return f"Erro na geração: {str(e)}"

## FUNÇÃO PRINCIPAL

In [13]:
def main(input_file_index=0, temperature=0.8):
    # Configuração
    config = ModelConfig()
    logger = Logger()

    logger.info("🚀 Iniciando MiniGPT v2.0...")
    logger.info(f"Configurações: batch_size={config.batch_size}, block_size={config.block_size}")

    # Configuração de reprodutibilidade
    torch.manual_seed(config.seed)

    # Processamento de dados
    logger.info("📚 Carregando e processando dados...")
    processor = TextDataProcessor(config, logger)
    train_data, val_data = processor.load_and_process_data(input_file_index)
    data_loader = DataLoader(train_data, val_data, config)

    # Criação do modelo
    logger.info("🧠 Criando modelo...")
    model = MiniGPT(config, processor.vocab_size)
    model = model.to(config.device)

    # Treinamento
    logger.info("🎯 Iniciando treinamento...")
    trainer = ModelTrainer(model, data_loader, config, logger)
    training_history = trainer.train()

    # Geração de texto
    generator = TextGenerator(model, processor, config, logger)

    # Prompts alinhados com o corpus de treinamento
    prompts = [
        "O que é acne?",
        "Principais causas da Acne",
        "O que é eczema",
        "O que é psoaríase",
        "O que é rosácea",
        "O que é vitiligo",
        "Como tratar eczema",
        "Rosácea causas"
    ]

    logger.info("\n🎨 === EXEMPLOS DE GERAÇÃO DE TEXTO ===")
    for i, prompt in enumerate(prompts, 1):
        try:
            generated_text = generator.generate_text(
                prompt, 
                max_new_tokens=200, 
                temperature=temperature
            )
            
            print(f"\n{'='*80}")
            print(f"📝 EXEMPLO {i}/8")
            print(f"{'='*80}")
            print(f"🔤 Prompt: '{prompt}'")
            print(f"{'─'*80}")
            print(f"🤖 Texto Gerado:")
            print(generated_text)
            print(f"{'='*80}")
            
        except Exception as e:
            logger.error(f"Erro ao gerar texto para prompt '{prompt}': {e}")

    # Estatísticas finais
    final_train_loss = training_history['train_loss'][-1] if training_history['train_loss'] else 'N/A'
    final_val_loss = training_history['val_loss'][-1] if training_history['val_loss'] else 'N/A'

    logger.info(f"\n📊 === ESTATÍSTICAS FINAIS ===")
    logger.info(f"✅ Treinamento concluído com sucesso!")
    logger.info(f"📈 Perda final de treino: {final_train_loss}")
    logger.info(f"📉 Perda final de validação: {final_val_loss}")
    logger.info(f"🔢 Parâmetros do modelo: {model.count_parameters()/1e6:.2f}M")
    logger.info(f"📖 Tamanho do vocabulário: {processor.vocab_size}")

### Textos curtos

In [14]:
main(input_file_index=0)

2025-11-16 23:31:54,299 - MiniGPT - INFO - 🚀 Iniciando MiniGPT v2.0...
2025-11-16 23:31:54,299 - MiniGPT - INFO - Configurações: batch_size=32, block_size=64
2025-11-16 23:31:54,301 - MiniGPT - INFO - 📚 Carregando e processando dados...
2025-11-16 23:31:54,302 - MiniGPT - INFO - Texto carregado: 7470 caracteres
2025-11-16 23:31:54,303 - MiniGPT - INFO - Vocabulário criado com 51 caracteres únicos
2025-11-16 23:31:54,303 - MiniGPT - INFO - Amostra do vocabulário: 
 ,-.ACDEFMNOPTabcdefghijlmnop...
2025-11-16 23:31:54,304 - MiniGPT - INFO - Dados divididos: 6723 treino, 747 validação
2025-11-16 23:31:54,305 - MiniGPT - INFO - 🧠 Criando modelo...
2025-11-16 23:31:54,404 - MiniGPT - INFO - 🎯 Iniciando treinamento...
2025-11-16 23:31:55,154 - MiniGPT - INFO - Iniciando treinamento...
2025-11-16 23:31:55,155 - MiniGPT - INFO - Parâmetros do modelo: 1.21M
2025-11-16 23:31:55,156 - MiniGPT - INFO - Dispositivo: cuda
2025-11-16 23:32:01,475 - MiniGPT - INFO - Iteração 0: perda treino 3.9719, per


📝 EXEMPLO 1/8
🔤 Prompt: 'O que é acne?'
────────────────────────────────────────────────────────────────────────────────
🤖 Texto Gerado:
O que é acne fatores como estresse, uso inadequado de cosméticos e alimentação.



A acne é identificada pela presença de lesões inflamatórias na face como cravos pretos ou brancos, espinhas vermelhas e pústulas.


2025-11-16 23:38:12,561 - MiniGPT - INFO - Gerando texto com prompt: 'O que é eczema'



📝 EXEMPLO 2/8
🔤 Prompt: 'Principais causas da Acne'
────────────────────────────────────────────────────────────────────────────────
🤖 Texto Gerado:
Principais causas da Acnes incluem estresse emocional, infecções, lesões na pele, clima frio e seco, consumo excessivo de álcoool, tabagismo, obesidade e uso de medicamentos como anti-inflamatórios e betabloqueadores.



A ro


2025-11-16 23:38:15,944 - MiniGPT - INFO - Gerando texto com prompt: 'O que é psoaríase'


...


📝 EXEMPLO 6/8
🔤 Prompt: 'O que é vitiligo'
────────────────────────────────────────────────────────────────────────────────
🤖 Texto Gerado:
O que é vitiligo envolvem predisposição genética, resposta autoimune que destrói os melanócitos, fatores ambientais como traumas físicos ou queimaduras solares, estresse emocional intenso e possivelmente deficiências


2025-11-16 23:38:30,278 - MiniGPT - INFO - Gerando texto com prompt: 'Rosácea causas'
2025-11-16 23:38:30,279 - MiniGPT - ERROR - Caractere 'R' não encontrado no vocabulário



📝 EXEMPLO 7/8
🔤 Prompt: 'Como tratar eczema'
────────────────────────────────────────────────────────────────────────────────
🤖 Texto Gerado:
Como tratar eczema inclui hidratação intensa com cremes emolientes, uso de corticosteroides tópicos para controlar a inflamação, anti-histamínicos para coceira, evitar gatilhos conhecidos e em casos graves imunomodulad


2025-11-16 23:38:33,799 - MiniGPT - INFO - 
📊 === ESTATÍSTICAS FINAIS ===
2025-11-16 23:38:33,799 - MiniGPT - INFO - ✅ Treinamento concluído com sucesso!
2025-11-16 23:38:33,799 - MiniGPT - INFO - 📈 Perda final de treino: 0.12809927761554718
2025-11-16 23:38:33,800 - MiniGPT - INFO - 📉 Perda final de validação: 3.149897813796997
2025-11-16 23:38:33,801 - MiniGPT - INFO - 🔢 Parâmetros do modelo: 1.21M
2025-11-16 23:38:33,801 - MiniGPT - INFO - 📖 Tamanho do vocabulário: 51



📝 EXEMPLO 8/8
🔤 Prompt: 'Rosácea causas'
────────────────────────────────────────────────────────────────────────────────
🤖 Texto Gerado:
osácea causas exatas da rosácea envolvem predisposição genética, resposta autoimune que destrói os melanócitos, fatores ambientais como traumas na pele, infecções estreptocócicas, estresse, uso de certos medicamen


### Textos longos

In [16]:
main(input_file_index=1)

2025-11-16 23:42:46,019 - MiniGPT - INFO - 🚀 Iniciando MiniGPT v2.0...
2025-11-16 23:42:46,019 - MiniGPT - INFO - Configurações: batch_size=32, block_size=64
2025-11-16 23:42:46,020 - MiniGPT - INFO - 📚 Carregando e processando dados...
2025-11-16 23:42:46,021 - MiniGPT - INFO - Texto carregado: 3019 caracteres
2025-11-16 23:42:46,021 - MiniGPT - INFO - Vocabulário criado com 46 caracteres únicos
2025-11-16 23:42:46,022 - MiniGPT - INFO - Amostra do vocabulário: 
 ,-.ACEFOSabcdefghijlmnopqrst...
2025-11-16 23:42:46,023 - MiniGPT - INFO - Dados divididos: 2717 treino, 302 validação
2025-11-16 23:42:46,024 - MiniGPT - INFO - 🧠 Criando modelo...
2025-11-16 23:42:46,113 - MiniGPT - INFO - 🎯 Iniciando treinamento...
2025-11-16 23:42:46,115 - MiniGPT - INFO - Iniciando treinamento...
2025-11-16 23:42:46,116 - MiniGPT - INFO - Parâmetros do modelo: 1.21M
2025-11-16 23:42:46,117 - MiniGPT - INFO - Dispositivo: cuda
2025-11-16 23:42:52,245 - MiniGPT - INFO - Iteração 0: perda treino 3.7966, per


📝 EXEMPLO 1/8
🔤 Prompt: 'O que é acne?'
────────────────────────────────────────────────────────────────────────────────
🤖 Texto Gerado:
O que é acne de tempapel relevante em seu desencadeamento. O diagnóstico do eczema baseia-se na observação dos sintomas e histórico do paciente, sendo frequente em crianças, mas podendo persistir ou surgir na vid


2025-11-16 23:46:37,857 - MiniGPT - INFO - Gerando texto com prompt: 'O que é eczema'



📝 EXEMPLO 2/8
🔤 Prompt: 'Principais causas da Acne'
────────────────────────────────────────────────────────────────────────────────
🤖 Texto Gerado:
rincipais causas da Acne idaturações climáticas desempenham papel relevante em seu desencadeamento. O diagnóstico do eczema baseia-se na observação dos sintomas e histórico do paciente, sendo frequente em crianças, mas poden


2025-11-16 23:46:41,213 - MiniGPT - INFO - Gerando texto com prompt: 'O que é psoaríase'


...


📝 EXEMPLO 6/8
🔤 Prompt: 'O que é vitiligo'
────────────────────────────────────────────────────────────────────────────────
🤖 Texto Gerado:
O que é vitiligo compreende uma desordem caracterizada pela perda subsequente da pigmentação cutânea, devido à destruição dos melanócitos. O quadro se manifesta pelo surgimento de manchas brancas na pele, sendo o ros


2025-11-16 23:46:54,780 - MiniGPT - INFO - Gerando texto com prompt: 'Rosácea causas'
2025-11-16 23:46:54,780 - MiniGPT - ERROR - Caractere 'R' não encontrado no vocabulário



📝 EXEMPLO 7/8
🔤 Prompt: 'Como tratar eczema'
────────────────────────────────────────────────────────────────────────────────
🤖 Texto Gerado:
Como tratar eczema, também chamado dermatite atópica, é uma doença inflamatória recorrente que manifesta-se através de áreas de pele avermelhada, coceira intensa e descamação. Fatores hereditários, exposição a alérgeno


2025-11-16 23:46:58,141 - MiniGPT - INFO - 
📊 === ESTATÍSTICAS FINAIS ===
2025-11-16 23:46:58,141 - MiniGPT - INFO - ✅ Treinamento concluído com sucesso!
2025-11-16 23:46:58,141 - MiniGPT - INFO - 📈 Perda final de treino: 0.09703146666288376
2025-11-16 23:46:58,142 - MiniGPT - INFO - 📉 Perda final de validação: 3.942631721496582
2025-11-16 23:46:58,142 - MiniGPT - INFO - 🔢 Parâmetros do modelo: 1.21M
2025-11-16 23:46:58,143 - MiniGPT - INFO - 📖 Tamanho do vocabulário: 46



📝 EXEMPLO 8/8
🔤 Prompt: 'Rosácea causas'
────────────────────────────────────────────────────────────────────────────────
🤖 Texto Gerado:
osácea causas dempenham papel relevante em seu desencadeamento. O diagnóstico do eczema baseia-se na observação dos sintomas e histórico do paciente, sendo frequente em crianças, mas podendo persistir ou surgir na


### Perguntas e respostas

In [17]:
main(input_file_index=2)

2025-11-16 23:46:58,151 - MiniGPT - INFO - 🚀 Iniciando MiniGPT v2.0...
2025-11-16 23:46:58,152 - MiniGPT - INFO - Configurações: batch_size=32, block_size=64
2025-11-16 23:46:58,153 - MiniGPT - INFO - 📚 Carregando e processando dados...
2025-11-16 23:46:58,154 - MiniGPT - INFO - Texto carregado: 8930 caracteres
2025-11-16 23:46:58,155 - MiniGPT - INFO - Vocabulário criado com 56 caracteres únicos
2025-11-16 23:46:58,155 - MiniGPT - INFO - Amostra do vocabulário: 
 ,-.:?ACDEFMNOPQRTVabcdefghij...
2025-11-16 23:46:58,156 - MiniGPT - INFO - Dados divididos: 8037 treino, 893 validação
2025-11-16 23:46:58,157 - MiniGPT - INFO - 🧠 Criando modelo...
2025-11-16 23:46:58,189 - MiniGPT - INFO - 🎯 Iniciando treinamento...
2025-11-16 23:46:58,191 - MiniGPT - INFO - Iniciando treinamento...
2025-11-16 23:46:58,192 - MiniGPT - INFO - Parâmetros do modelo: 1.21M
2025-11-16 23:46:58,193 - MiniGPT - INFO - Dispositivo: cuda
2025-11-16 23:47:04,339 - MiniGPT - INFO - Iteração 0: perda treino 4.0460, per


📝 EXEMPLO 1/8
🔤 Prompt: 'O que é acne?'
────────────────────────────────────────────────────────────────────────────────
🤖 Texto Gerado:
O que é acne?
Resposta: A acne é uma condição inflamatória da pele que ocorre quando os folículos pilossebáceos ficam obstruídos por óleo e células mortas. Caracteriza-se por cravos, espinhas, pústulas e, em casos


2025-11-16 23:50:51,634 - MiniGPT - INFO - Gerando texto com prompt: 'O que é eczema'



📝 EXEMPLO 2/8
🔤 Prompt: 'Principais causas da Acne'
────────────────────────────────────────────────────────────────────────────────
🤖 Texto Gerado:
Principais causas da Acne?
Resposta: As principais causas da acne incluem alterações hormonais especialmente na puberdade, produção excessiva de sebo pelas glândulas sebáceas, proliferação de bactérias, predisposição genética


2025-11-16 23:50:55,009 - MiniGPT - INFO - Gerando texto com prompt: 'O que é psoaríase'


...


📝 EXEMPLO 6/8
🔤 Prompt: 'O que é vitiligo'
────────────────────────────────────────────────────────────────────────────────
🤖 Texto Gerado:
O que é vitiligo?
Resposta: O vitiligo é uma doença autoimune que causa perda da pigmentação da pele, resultando em manchas brancas. Ocorre devido à destruição dos melanócitos, células responsáveis pela produção de m


2025-11-16 23:51:08,549 - MiniGPT - INFO - Gerando texto com prompt: 'Rosácea causas'



📝 EXEMPLO 7/8
🔤 Prompt: 'Como tratar eczema'
────────────────────────────────────────────────────────────────────────────────
🤖 Texto Gerado:
Como tratar eczema?
Resposta: O tratamento para vitiligo inclui corticosteroides tópicos, inibidores de calcineurina, fatores imunológicos, alergias ambientais, exposição a iritantes como sabões e detergentes, mudanças


2025-11-16 23:51:11,972 - MiniGPT - INFO - 
📊 === ESTATÍSTICAS FINAIS ===
2025-11-16 23:51:11,972 - MiniGPT - INFO - ✅ Treinamento concluído com sucesso!
2025-11-16 23:51:11,973 - MiniGPT - INFO - 📈 Perda final de treino: 0.13190113008022308
2025-11-16 23:51:11,973 - MiniGPT - INFO - 📉 Perda final de validação: 2.7712907791137695
2025-11-16 23:51:11,974 - MiniGPT - INFO - 🔢 Parâmetros do modelo: 1.21M
2025-11-16 23:51:11,974 - MiniGPT - INFO - 📖 Tamanho do vocabulário: 56



📝 EXEMPLO 8/8
🔤 Prompt: 'Rosácea causas'
────────────────────────────────────────────────────────────────────────────────
🤖 Texto Gerado:
Rosácea causas exatas da rosácea?
Resposta: As causas exatas da rosácea não são totalmente conhecidas, mas acredita-se que envolvam fatores genéticos, anormalidades vasculares, reação inflamatória, ácaros Demodex, 
